In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import os
from collections import defaultdict
import librosa
import pandas as pd
import numpy as np
import pickle

# 폴더별 파일 개수 확인
def count_files_in_folders(label_paths):
    file_counts = {}
    for label, folder_path in label_paths.items():
        try:
            file_counts[label] = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
        except FileNotFoundError:
            print(f"Folder not found: {folder_path}")
            file_counts[label] = 0
    return file_counts

In [43]:
# 데이터 경로와 라벨 매핑
path = '/content/drive/MyDrive/Aivle_빅프/china_sep/'
labels = {
    'awake': path + 'sep_awake/',
    'diaper': path + 'sep_diaper/',
    'hug': path + 'sep_hug/',
    'hungry': path + 'sep_hungry/',
    'sleepy': path + 'sep_sleepy/',
    # 'silence'  : path + 'sep_silence/',
    'uncomfortable': path + 'sep_uncomfortable/'
}

# 파일 개수 출력
file_counts = count_files_in_folders(labels)
for label, count in file_counts.items():
    print(f"{label}: {count} files")

awake: 740 files
diaper: 620 files
hug: 868 files
hungry: 787 files
sleepy: 710 files
uncomfortable: 671 files


In [11]:
def feature_extractor_main(audio_file_path):
    audio,sample_rate = librosa.load(audio_file_path)
    mffcs_features = librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=80)
    mfccs_scaled_features = np.mean(mffcs_features.T,axis=0)
    return mfccs_scaled_features

In [12]:
def feature_extractor(labels):
    extracted_features = []

    for label, folder_path in labels.items():
        if not os.path.exists(folder_path):  # 폴더가 존재하는지 확인
            print(f"Warning: {folder_path} does not exist. Skipping...")
            continue

        for file in os.listdir(folder_path):
            if file.endswith('.wav') or file.endswith('.mp3'):  # 오디오 파일만 처리
                audio_file_path = os.path.join(folder_path, file)
                mfccs_scaled_features = feature_extractor_main(audio_file_path)
                extracted_features.append([label, mfccs_scaled_features])  # label을 사용

    return extracted_features

In [13]:
extracted_features = feature_extractor(labels)

In [14]:
extracted_features[:1]

[['awake',
  array([-2.1912604e+02,  6.4258621e+01, -9.9105484e+01,  1.5254613e+01,
         -7.7594166e+00,  2.5632517e+00, -2.3923824e+01,  2.7655413e+01,
         -1.6203617e+01, -1.0088612e+01, -2.1863863e+00, -1.3037782e+01,
          1.5829412e+01, -6.0526700e+00,  8.2750263e+00,  6.1734428e+00,
          1.1484527e+01,  5.7044392e+00,  2.0332286e+00,  1.2279929e+00,
          4.1493611e+00,  1.1781603e+00, -6.1528311e+00,  2.4399145e+00,
          3.7671363e-01,  8.5591154e+00,  3.9774540e-01,  3.6729095e+00,
         -4.4767013e-01, -1.2862679e+00,  2.9167469e+00, -1.5548818e+00,
          3.0287328e+00, -2.3453650e+00, -1.8099533e+00, -1.3441091e+00,
          1.0065087e+00, -2.4727715e-02,  3.0315552e+00, -1.1506457e+00,
         -1.9355146e+00,  1.4545548e+00, -1.6195394e+00,  8.4893376e-01,
         -2.8502107e+00,  4.4833437e-01, -9.2230600e-01, -1.0204626e+00,
          3.3352759e-01,  1.3167188e+00,  3.0027282e+00,  6.2273490e-01,
         -6.5374464e-01, -1.5914049e+00,

In [15]:
# ✅ 리스트 데이터를 DataFrame 형태로 변환
features_and_class_data = pd.DataFrame(
    [(row[0], *row[1]) for row in extracted_features],  # 리스트를 펼쳐서 열로 변환
    columns=['Class'] + [f'Feature{i+1}' for i in range(80)]  # 열 이름 설정
)

In [16]:
print(features_and_class_data.shape)
print(features_and_class_data['Class'].value_counts())
features_and_class_data.tail()

(4366, 81)
Class
hug              863
hungry           782
awake            735
sleepy           705
uncomfortable    666
diaper           615
Name: count, dtype: int64


,Class,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,...,Feature71,Feature72,Feature73,Feature74,Feature75,Feature76,Feature77,Feature78,Feature79,Feature80
4361,uncomfortable,-531.082581,32.492184,-14.237103,8.529375,-10.101477,3.352972,-7.291878,4.121451,-6.097635,...,-0.062096,-0.678498,-1.376045,-0.510742,-0.945488,-0.747567,-0.233507,-0.166027,-0.743914,-0.189624
4362,uncomfortable,-215.286484,82.940216,-81.491272,-0.075469,-32.963200,19.884562,-19.158285,18.683563,9.304358,...,-0.232470,-1.088428,0.117694,-1.527397,-0.768751,-1.633790,-0.802958,-0.730500,-0.411943,-1.242277
4363,uncomfortable,-255.302811,129.023026,-33.895962,-21.730247,-42.937862,12.188851,-24.643112,3.004463,5.830189,...,-1.605092,-1.622476,-0.826004,-1.456343,-0.244447,-1.499420,-0.802523,-0.313644,-1.457552,-0.360399
4364,uncomfortable,-430.428925,72.617996,-44.270462,-37.542095,-29.276707,-3.763151,-31.303881,12.178646,-10.198870,...,0.557408,1.310208,-2.682799,0.151087,-2.568694,-0.796867,0.066391,-1.590955,0.581671,0.491748
4365,uncomfortable,-426.333130,71.879570,-31.483360,-38.036186,-12.857313,-3.285716,-33.218399,18.148367,-6.056257,...,-1.743399,-0.442396,-1.192863,-1.471666,-1.043113,-1.362227,-1.178004,0.499508,-1.624838,0.312339


In [17]:
features_and_class_data.to_csv('/content/drive/MyDrive/Infant_crying_finetuning/data/china_mfcc80.csv')

# 시도 2 : 전처리x + mfcc 240

In [24]:
features_and_class_data = pd.read_csv('/content/drive/MyDrive/Infant_crying_finetuning/data/china_mfcc80.csv', index_col = 0)
features_and_class_data = features_and_class_data[features_and_class_data['Class'] != 'uncomfortable']

print(features_and_class_data.shape)
print(features_and_class_data['Class'].value_counts())
features_and_class_data.tail()

(3700, 81)
Class
hug       863
hungry    782
awake     735
sleepy    705
diaper    615
Name: count, dtype: int64


,Class,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,...,Feature71,Feature72,Feature73,Feature74,Feature75,Feature76,Feature77,Feature78,Feature79,Feature80
3695,sleepy,-291.79460,88.46475,-47.410347,14.908660,-21.524956,9.830672,-18.296726,8.044330,0.455599,...,-0.715852,-0.286895,0.119725,-0.522526,-1.386731,-1.607595,-0.933362,-1.786232,-0.172259,0.068086
3696,sleepy,-380.90630,76.76793,-54.934920,4.756350,-29.341793,0.959860,-27.026308,20.144426,0.336655,...,-2.826862,-1.953026,-0.036875,-1.245380,-2.305145,-0.382267,-1.946730,0.144081,-1.034499,-1.768405
3697,sleepy,-274.23508,72.35922,-72.749460,-1.115077,-27.044764,14.480275,-32.686577,13.537096,-5.723475,...,0.630533,-1.840168,0.845064,-1.902651,1.322880,-1.034121,-1.311601,-0.582020,-1.250467,-0.807014
3698,sleepy,-342.35025,111.84306,-52.431816,30.890470,-18.445557,18.716673,-14.860747,27.668945,1.221442,...,-0.720423,-0.740181,0.736207,-1.163530,1.298228,-0.382690,-0.177731,-0.557360,-0.788667,-0.277677
3699,sleepy,-393.46290,133.38211,-35.072567,28.322233,-11.584337,20.066397,2.551679,26.187706,-1.178117,...,0.617834,-1.893668,-0.436809,-1.752981,-0.574052,-0.316598,-0.914424,-0.879232,-0.546627,-0.847065


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [26]:
# ✅ 데이터 준비
# X = X_expanded_mfcc  # 기존 Feature 데이터
X = features_and_class_data.drop('Class', axis=1)
y = features_and_class_data['Class']

# ✅ StandardScaler 적용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # 새로운 데이터로 학습 후 변환

# ✅ LabelEncoder 적용
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(y)  # 클래스 이름을 정수로 변환

# ✅ One-hot Encoding 적용 (모델의 클래스 개수에 맞춤)
num_classes = len(label_encoder.classes_)  # 새로운 데이터에서의 클래스 개수
y_one_hot = to_categorical(Y, num_classes=num_classes)

# ✅ 변환 후 확인
print(f"X_scaled shape: {X_scaled.shape}")  # (샘플 수, Feature 개수)
print(f"y shape: {y_one_hot.shape}")  # (샘플 수, num_classes)
print(f"클래스 목록: {label_encoder.classes_}")  # 변환된 클래스 목록 출력

# ✅ StandardScaler와 LabelEncoder 저장
with open("./second_model_scaler.pkl", "wb") as scaler_file:
    pickle.dump(scaler, scaler_file)

with open("./second_model_label_encoder.pkl", "wb") as encoder_file:
    pickle.dump(label_encoder, encoder_file)

print("✅ Scaler와 LabelEncoder가 .pkl 파일로 저장되었습니다.")

X_scaled shape: (3700, 80)
y shape: (3700, 5)
클래스 목록: ['awake' 'diaper' 'hug' 'hungry' 'sleepy']
✅ Scaler와 LabelEncoder가 .pkl 파일로 저장되었습니다.


In [27]:
# ✅ Label Encoding 매핑 결과 확인
label_mapping = {index: class_name for index, class_name in enumerate(label_encoder.classes_)}

# ✅ 매핑 결과 출력
print("Label Encoding 매핑 결과:")
for key, value in label_mapping.items():
    print(f"{key} → {value}")

Label Encoding 매핑 결과:
0 → awake
1 → diaper
2 → hug
3 → hungry
4 → sleepy


In [28]:
y_one_hot[:2]

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [29]:
X_scaled

array([[ 1.25432617, -0.41644698, -1.75618599, ...,  0.5500836 ,
        -0.22012202,  0.76687198],
       [ 1.08406669, -0.5469588 , -1.49651514, ..., -0.14971607,
        -0.50966118,  0.14947337],
       [ 1.06818251, -0.57677486, -1.19613016, ..., -0.4498768 ,
        -1.26257322, -0.21504642],
       ...,
       [ 0.59727585, -0.11392357, -0.71781116, ..., -0.03375481,
        -1.13607863, -0.52309446],
       [-0.21484307,  1.36063212,  0.08266353, ..., -0.00261782,
        -0.65959587,  0.09036526],
       [-0.8242455 ,  2.16502522,  0.76658335, ..., -0.40903524,
        -0.40986066, -0.56950961]])

In [46]:
from sklearn.model_selection import train_test_split

# 1차 분할: Train(80%) / Test(20%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_one_hot, test_size=0.2, random_state=42)

# 2차 분할: Train(80%) → Train(70%) / Validation(10%)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# 출력 확인
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_valid shape: {X_valid.shape}')
print(f'y_valid shape: {y_valid.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (2368, 80)
y_train shape: (2368, 5)
X_valid shape: (592, 80)
y_valid shape: (592, 5)
X_test shape: (740, 80)
y_test shape: (740, 5)


In [32]:
import numpy as np
from sklearn.model_selection import train_test_split

# 원본 데이터의 인덱스 생성
indices = np.arange(len(X_scaled))

# 1차 분할: Train(80%) / Test(20%)
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
    X_scaled, y_one_hot, indices, test_size=0.2, random_state=42
)

# 2차 분할: Train(80%) → Train(70%) / Validation(10%)
X_train, X_valid, y_train, y_valid, train_idx, valid_idx = train_test_split(
    X_train, y_train, train_idx, test_size=0.2, random_state=42, stratify=y_train
)

# 인덱스 확인
# print(f'X_train indices: {train_idx}')
# print(f'X_valid indices: {valid_idx}')
print(f'X_test indices: {test_idx}')


X_test indices: [1042 2460  315 3523  811  321  402  744 2924 3238  351 3623  358  291
 1041  486 2948  990 2452  366 2202 2350 1511 1830 2645 1288 2584 1465
 2936  605 2743  752 1204  538  463   32  534 2594 1509 1292  964   93
 2862  415 2091   14 3538  354  120 2154 3012 2399 1417 2821 2830 1670
 2533 3568 1171  240 1084  478 2687 2302  864 2369  229   17 1467  211
 2019 3138  903 3214  196 2866 3660  594 2195  109 2895  952 1801 1270
  879  925 1741 1407 1074 1018  393  270 2245 2943  387 3129  450 1829
 1234  149 1233 1188  179 3368 1117  443  184 2014  298 2664  746 1770
  564 1334   52 3510 3299   26 3600 1501  937  495  798  256  408  764
  985 3447 2562  839 2915 2406 1488 1406 2249  809 2513 3426  599 2292
 1554 1783 2127 2159 1391 1681 1582 1833  299 1775   33 2969 1226 2547
  969 2566 2678 1268 2900 1480 1474 2473 2176 3070 1110 2288 1795  554
 2662 3295 1842   70 2984  544 3237  511 3281  438 3010 3105 1883 3567
 1164  573 2542 2328  670 2247 3199 3691 2086  170 2246 2684 

In [52]:
X_test_indices = list(test_idx)
print(len(X_test_indices))

740


In [60]:
import os
import shutil

def count_files_in_folders_sorted(folder_dict):
    file_counts = {}
    all_files = []

    for label, folder_path in folder_dict.items():
        files = os.listdir(folder_path)
        files = [f for f in files if os.path.isfile(os.path.join(folder_path, f))]  # 파일만 카운트
        file_counts[label] = len(files)
        all_files.extend([(os.path.join(folder_path, f), label) for f in files])

    return file_counts, all_files

# 데이터 경로와 라벨 매핑
path = '/content/drive/MyDrive/Aivle_빅프/china_sep/'
labels = {
    'awake': path + 'sep_awake/',
    'diaper': path + 'sep_diaper/',
    'hug': path + 'sep_hug/',
    'hungry': path + 'sep_hungry/',
    'sleepy': path + 'sep_sleepy/'
}

# 파일 개수와 파일 리스트 얻기
file_counts, all_files = count_files_in_folders_sorted(labels)

test_audio_base_path = "/content/drive/MyDrive/test_audio"
os.makedirs(test_audio_base_path, exist_ok=True)

# 라벨별 폴더 생성
for label_name in labels.keys():
    os.makedirs(os.path.join(test_audio_base_path, label_name), exist_ok=True)

X_test_indices = list(test_idx)

idxxxx = 0
# 파일을 인덱스 순서대로 저장
for idx, (audio_path, label) in enumerate(all_files):
    if idx in X_test_indices:  # X_test 인덱스와 일치하는 경우
        destination_path = os.path.join(test_audio_base_path, label)
        shutil.copy(audio_path, destination_path)
        print(f"Saved: {audio_path} -> {destination_path}")
        idxxxx += 1
print(idxxxx)

Saved: /content/drive/MyDrive/Aivle_빅프/china_sep/sep_awake/664_snippet_2.npy -> /content/drive/MyDrive/test_audio/awake
Saved: /content/drive/MyDrive/Aivle_빅프/china_sep/sep_awake/awake_0_snippet_1.wav -> /content/drive/MyDrive/test_audio/awake
Saved: /content/drive/MyDrive/Aivle_빅프/china_sep/sep_awake/awake_10_snippet_1.wav -> /content/drive/MyDrive/test_audio/awake
Saved: /content/drive/MyDrive/Aivle_빅프/china_sep/sep_awake/awake_100_snippet_1.wav -> /content/drive/MyDrive/test_audio/awake
Saved: /content/drive/MyDrive/Aivle_빅프/china_sep/sep_awake/awake_102_snippet_1.wav -> /content/drive/MyDrive/test_audio/awake
Saved: /content/drive/MyDrive/Aivle_빅프/china_sep/sep_awake/awake_104_snippet_3.wav -> /content/drive/MyDrive/test_audio/awake
Saved: /content/drive/MyDrive/Aivle_빅프/china_sep/sep_awake/awake_105_snippet_4.wav -> /content/drive/MyDrive/test_audio/awake
Saved: /content/drive/MyDrive/Aivle_빅프/china_sep/sep_awake/awake_106_snippet_1.wav -> /content/drive/My

In [61]:
# 데이터 경로와 라벨 매핑
path = '/content/drive/MyDrive/test_audio/'
labels = {
    'awake': path + 'awake/',
    'diaper': path + 'diaper/',
    'hug': path + 'hug/',
    'hungry': path + 'hungry/',
    'sleepy': path + 'sleepy/'
}

# 파일 개수 출력
file_counts = count_files_in_folders(labels)
for label, count in file_counts.items():
    print(f"{label}: {count} files")

awake: 167 files
diaper: 124 files
hug: 166 files
hungry: 155 files
sleepy: 128 files


In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from datetime import datetime

In [48]:
num_labels = y_train.shape[1]
input_shape = X_train.shape[1]

print(num_labels)
print(input_shape)

5
80


In [49]:
model = Sequential()
model.add(Dense(256, input_shape=(input_shape,)))
model.add(BatchNormalization())
model.add(Activation(LeakyReLU(alpha=0.1)))
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation(LeakyReLU(alpha=0.2)))
model.add(Dropout(0.2))

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation(LeakyReLU(alpha=0.1)))
model.add(Dropout(0.4))

model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation(LeakyReLU(alpha=0.1)))
model.add(Dropout(0.2))

model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 256)                 │          20,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 32)                  │              

 Total params: 83,077 (324.52 KB)

 Trainable params: 81,861 (319.77 KB)

 Non-trainable params: 1,216 (4.75 KB)

In [50]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

# ✅ Callbacks 설정 (ModelCheckpoint + EarlyStopping)
checkpointer = ModelCheckpoint(filepath='./second_model_sim2_mfcc80.keras', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# ✅ 학습 파라미터
num_epochs = 300
num_batch_size = 32

# ✅ 학습 시작 시간 기록
start = datetime.now()

# ✅ 모델 학습 (EarlyStopping 추가)
history = model.fit(
    X_train, y_train,
    batch_size=num_batch_size,
    epochs=num_epochs,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpointer, early_stopping]
)

# ✅ 학습 시간 출력
duration = datetime.now() - start
print(f"The model training takes {duration}")

Epoch 1/300
71/74 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2087 - loss: 1.8950
Epoch 1: val_loss improved from inf to 1.56662, saving model to ./second_model_sim2_mfcc80.keras
74/74 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - accuracy: 0.2094 - loss: 1.8910 - val_accuracy: 0.3007 - val_loss: 1.5666
Epoch 2/300
72/74 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2210 - loss: 1.7215
Epoch 2: val_loss improved from 1.56662 to 1.51965, saving model to ./second_model_sim2_mfcc80.keras
74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.2214 - loss: 1.7202 - val_accuracy: 0.3429 - val_loss: 1.5196
Epoch 3/300
70/74 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2568 - loss: 1.6197
Epoch 3: val_loss improved from 1.51965 to 1.47476, saving model to ./second_model_sim2_mfcc80.keras
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2564 - loss: 1.6196 - val_accuracy: 0.3767 - val_loss: 1.4748
Epoch 4/300
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2877 - loss: 1.5749
Epoch 4: v

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# ✅ 학습 과정에서 기록된 데이터 가져오기
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# ✅ EarlyStopping이 적용된 경우, 실제 학습된 Epoch 범위 설정
epochs_range = range(1, len(train_loss) + 1)

# ✅ Loss 시각화
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()

# ✅ Accuracy 시각화
plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training & Validation Accuracy')
plt.legend()

# ✅ 그래프 표시
plt.show()


In [ ]:
y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)
y_actual_classes = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_actual_classes, y_pred_classes)
print("Accuracy:", accuracy)

conf_matrix = confusion_matrix(y_actual_classes, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
from sklearn.metrics import classification_report

# ✅ 예측값 변환
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_actual_classes = np.argmax(y_test, axis=1)

# ✅ Classification Report 출력 (Precision, Recall, F1-score, Accuracy 포함)
print(classification_report(y_actual_classes, y_pred_classes, digits=4))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# ✅ 예측값 변환
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_actual_classes = np.argmax(y_test, axis=1)

# ✅ 혼동 행렬 생성
conf_matrix = confusion_matrix(y_actual_classes, y_pred_classes)

# ✅ 기본적인 시각화
plt.figure(figsize=(5, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")

# ✅ 기본 라벨 설정
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")

plt.show()


In [ ]:
n_classes = y_test.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for class %d' % (roc_auc[i], i))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()